link do repozytorium: [link](https://gitlab-stud.elka.pw.edu.pl/jmarcows/ium_22z_projekt "https://gitlab-stud.elka.pw.edu.pl/jmarcows/ium_22z_projekt")

## **Proces budowy modeli**

### **Model bazowy**

In [1]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

Naszym modelem bazowym jest model `LinearRegression` wbudowany w pakiet sklearn. Zadaniem tego modelu jest obliczenie predykcji ilości odsłuchań na kolejny tydzień bazując na danych z ostatniych 4 tygodni. \
W pliku `microservice/models/base_model/base_model.ipynb` znajduje się skrypt przygotwujący dostępne dane do formatu danych wejściowych oraz kod źródłowy modelu.

#### **Format danych wejściowych modelu bazowego**
Próbka danych treningowych znajduje się poniżej. Przykład danych treningowych dla modelu bazowego można zobaczyć w pliku `data/training_data/base_model.json`
```py
[
    ...,
    {
        "track_id": "5LNiqEqpDc8TuqPy79kDBu",
        "play_count": 2,
        "play_count_week_1": 1,
        "play_count_week_2": 2,
        "play_count_week_3": 0,
        "play_count_week_4": 1
    },
    ...
]

```

W naszym modelu bazowym wejściem jest czteroelementowy wektor, gdzie każda wartość to ilość odsłuchań danego utworu w kolejnym tygodniu. \
Zważywszy na prostote modelu zdecydowaliśmy się na trenowanie modelu przed każdym generowaniem nowej playlisty. Dane wejściowe zawierają w sobie informacje o ilośći odsłuchań dla każdej piosenki sprzed ostatnich 4 tygodni.

### **Model docelowy**

In [2]:
import torch
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self,
                 num_inputs: int,
                 num_hidden1: int,
                 num_hidden2: int,
                 num_outputs: int) -> None:
        super().__init__()
        self.linear1 = nn.Linear(num_inputs, num_hidden1)
        self.linear2 = nn.Linear(num_hidden1, num_hidden2)
        self.linear3 = nn.Linear(num_hidden2, num_outputs)
        self.act_fn = nn.Tanh()

    def forward(self, x):
        x = self.linear1(x)
        x = self.act_fn(x)
        x = self.linear2(x)
        x = self.act_fn(x)
        x = self.linear3(x)
        return x

Model docelowy to sieć neuronowa zaimplementowana za pomocą bilbioteki `PyTorch`. W modelu znajdują się dwie warstwy ukryte o określonej ilości neuronów.
Zadaniem modelu jest obliczenie predykcji ilości odsłuchań na kolejny tydzień bazując na danych z ostatnich 4 tygodni oraz dodatkowych cechach piosenek (dzięki temu docelowo będzie można wygenerować top listę składającą się z 50 utworów o największej liczbie odsłuchań - należy jednak pamietać, że obejmuje ona okres 4 tygodni).
W pliku `microservice/models/adv_model/adv_model.py` znajduje się pełna implementacja modelu oraz jego treningu razem z przygotowaniem danych (uwaga: pliki były przenoszone po ich wykorzystaniu, a zatem poprawność ścieżek nie jest gwarantowana).

Wykres przedstawiający funkcję straty w trakcie treningu modelu:

![adv_model_losses.png](adv_model_losses.png "adv_model_losses.png")

Szczególnie warto zwrócić uwagę na fluktuacje wartości funkcji straty począwszy od około 1800 epoki - wynikają one z faktu, że na modelu została "wymuszona" zdolność generalizacji za pomocą opcji "weight decay" (współczynnika regularyzacji L2) użytego optymalizatora Adam. Dzięki temu model jest mniej podatny na zjawisko przeuczania.

### **Format danych wejściowych modelu docelowego**

Aby otrzymać dane wejściowe dla modelu (do uczenia / testowania) należy:
1. Pobrać dane: [link](https://wutwaw-my.sharepoint.com/:u:/g/personal/pawel_zawistowski_pw_edu_pl/EWKqnTFghGlHqHiIVjBDDGoBKdQw12isgMhdWI67Z4479w?e=SsiI7g "https://wutwaw-my.sharepoint.com/:u:/g/personal/pawel_zawistowski_pw_edu_pl/EWKqnTFghGlHqHiIVjBDDGoBKdQw12isgMhdWI67Z4479w?e=SsiI7g"),
2. Rozpakować pobrany plik zip i umieścić w nowo utworzonym folderze folder `data`,
3. Poszczególne pliki .jsonl umieścić w odpowiednich podfolderach wewnątrz folderu data, np. `data/data/sessions/sessions.jsonl`,
4. Uruchomić skrypt [data_parser.ipynb](https://gitlab-stud.elka.pw.edu.pl/jmarcows/ium_22z_projekt/-/blob/main/data_parser.ipynb "https://gitlab-stud.elka.pw.edu.pl/jmarcows/ium_22z_projekt/-/blob/main/data_parser.ipynb"),
5. Plik `unique_ids.json` umieścić w folderze `microservice/`.

Po wykonaniu tych kroków wszelkie operacje na danych będą wykonywane wewnątrz poszczególnych skryptów.

Próbka danych treningowych znajduje się poniżej. 
```py
[  # cała paczka danych
  [  # tydzień 1
    {  # utwór 1
      "track_id",
      "popularity",
      "duration_ms",
      "explicit",
      "danceability",
      "energy",
      "key",
      "loudness",
      "speechiness",
      "acousticness",
      "instrumentalness",
      "liveness",
      "valence",
      "tempo",
      "release_date_year",
      "release_date_week",
      "play_count",
      "likes",
      "number_of_skips"
    },
    ... # utwór 2, ..., utwór 4071 (liczba utworów, dla których liczba odtworzeń > 0 (na podstawie całego okresu objętego przez sessions.jsonl))
  ],
  ...  # tydzień 2, tydzień 3, tydzień 4 (dane z miesiąca wstecz)
]


```

## **Porównanie wyników**

### **Mikroserwis**

W repozytorium znajduje się folder `microservice`, w którym znajduje się implementacja mikroserwisu umożliwiającego serwowanie predykcji przy pomocy wybranego modelu. \
Przy implementacji została wykorzystana biblioteka `FastAPI` z racji tego, że jest ona szybka w obsłudze oraz pozwala na szybką skalowalność naszej aplikacji.

Cała aplikacja została skonteneryzowana za pomocą środowiska Docker. Aby uruchomić aplikację lokalnie należy wykonać komendę z wiersza lini poleceń, znajdując się w katalogu roboczym repozytorium.
```
docker compose up --build
``` 
Aby wyłączyć kontener należy wykonać:
```
docker compose down
```

### **Opis endpointów**
Dzięki integracji `Swagger UI` z `FastAPI` tworzona jest automatyczna dokumentacja wszystkich endpointów wystawionych na komunikację w naszej aplikacji. Aby się do niej dostać, należy po uruchomieniu kontenera wejść na URL: `localhost:8080/docs`

Zważając na przejrzystość raportu końcowego zdecydowaliśmy się na umieszczenie krótkiego opisu endpointów:
- **/base-model/predict** - POST, służący do generowania jednorazowej predykcji wartości odsłuchań piosenki, za pomocą modelu bazowego, na bazie danych wejściowych zawierających dane odsłuchań z ostatnich 4 tygodni zawartych w ciele zapytania HTTP

- **/base-model/predictions** - POST, generuje listę predykcji, za pomocą modelu bazowego, przyjmując listę piosenek w formacie danych wejściowych bazowego modelu (opis wyżej) w ciele zapytania HTTP w fformie pliku JSON

- **/model/predictions** - POST, generuje listę predykcji, za pomocą modelu docelowego, przyjmując listę piosenek w formacie danych wejsciowych modelu docelowego (opis wyżej) w ciele zapytania HTTP w formie pliku JSON

Aplikacja przetrzymuje wytrenowane modele kolejno w folderach:
- `microservice/models/adv_model/adv_model.tar` - dla modelu docelowego 
- `microservice/models/base_model/base_model.joblib` - dla modelu bazowego

Z racji na wielkość wytrenowanego modelu docelowego zdecydowaliśmy się na nieumieszczanie go w zdalnym repozytorium.\
Model można pobrać klikając w link: https://drive.google.com/file/d/1CfHCXFw5sJv3_L_wfz3bOvEwVosvBJ5g/view?usp=sharing.

## **Eksperymenty A/B**

Przed przystąpieniem do testów zakładamy, że model bazowy (oznaczony jako `Base model`) będzie gorszy (względem ustalonej podczas Etapu I metryki) od modelu docelowego (oznaczonego jako `Adv. model`).

In [3]:
import json
import os
from typing import List

import pandas as pd

Adv. model

In [4]:
dirname = os.path.abspath('')
input_path = os.path.join(dirname,
                          "microservice/"
                          "database/"
                          "adv_model/"
                          "2023-01-17_00-33-27/"
                          "input.json")
output_path = os.path.join(dirname,
                           "microservice/"
                           "database/"
                           "adv_model/"
                           "2023-01-17_00-33-27/"
                           "output.json")
ranking_path = os.path.join(dirname,
                            "microservice/"
                            "database/"
                            "adv_model/"
                            "2023-01-17_00-33-27/"
                            "ranking.json")

with open(input_path, 'r') as f:
    input_data = json.load(f)

data_weeks: List[pd.DataFrame] = []
for i, week in enumerate(input_data):
    if i == 0:
        continue
    df = pd.DataFrame(week)
    data_weeks.append(df)

with open(output_path, 'r') as f:
    output_data = json.load(f)


def generate_ranking(output_data: List[float],
                     data_weeks: List[pd.DataFrame]) -> None:
    columns_to_drop = ["track_id", "popularity", "duration_ms", "explicit",
                       "danceability", "energy", "key", "loudness",
                       "speechiness", "acousticness", "instrumentalness",
                       "liveness", "valence", "tempo", "release_date_year",
                       "release_date_week", "likes", "number_of_skips"]

    for i, df in enumerate(data_weeks):
        tmp_df = df.copy()
        tmp_df.drop(columns=columns_to_drop, inplace=True)
        tmp_df = tmp_df.values.tolist()
        tmp_df_2 = []
        for row in tmp_df:
            tmp_df_2.extend(row)
        data_weeks[i] = tmp_df_2

    for i in range(len(output_data)):
        output_data[i] = (output_data[i] + data_weeks[0][i] +
                          data_weeks[1][i] + data_weeks[2][i])

    ranking = sorted(range(len(output_data)),
                     key=lambda i: output_data[i])
    ranking.reverse()

    for i, idx in enumerate(ranking[:50]):
        print(f"{i}: {idx} - {round(output_data[idx])}")

    with open(ranking_path, 'w+') as f:
        f.write(json.dumps(ranking, indent=4))


generate_ranking(output_data, data_weeks)

0: 60 - 256
1: 454 - 243
2: 994 - 241
3: 3620 - 238
4: 3988 - 238
5: 1380 - 237
6: 2643 - 237
7: 4000 - 236
8: 1136 - 235
9: 2168 - 233
10: 1234 - 232
11: 1509 - 231
12: 1383 - 229
13: 2792 - 229
14: 1922 - 229
15: 3986 - 226
16: 1601 - 226
17: 462 - 226
18: 1639 - 226
19: 901 - 226
20: 834 - 225
21: 3189 - 225
22: 1272 - 225
23: 3854 - 225
24: 4020 - 224
25: 870 - 224
26: 2241 - 224
27: 1534 - 223
28: 263 - 223
29: 1111 - 223
30: 2699 - 222
31: 3344 - 222
32: 1231 - 221
33: 1097 - 221
34: 1732 - 221
35: 3266 - 220
36: 3437 - 220
37: 1157 - 219
38: 2778 - 219
39: 2382 - 218
40: 2913 - 218
41: 3521 - 218
42: 1959 - 218
43: 443 - 217
44: 2902 - 216
45: 3670 - 216
46: 1442 - 216
47: 1668 - 216
48: 4030 - 215
49: 1827 - 215


Base model

In [5]:
dirname = os.path.abspath('')
input_path = os.path.join(dirname,
                          "microservice/"
                          "database/"
                          "base_model/"
                          "2023-01-17_02-12-19/"
                          "input.json")
output_path = os.path.join(dirname,
                           "microservice/"
                           "database/"
                           "base_model/"
                           "2023-01-17_02-12-19/"
                           "output.json")
ranking_path = os.path.join(dirname,
                            "microservice/"
                            "database/"
                            "base_model/"
                            "2023-01-17_02-12-19/"
                            "ranking.json")
unique_ids_path = os.path.join(dirname,
                               "microservice/"
                               "unique_ids.json")

with open(input_path, 'r') as f:
    input_data = json.load(f)

with open(unique_ids_path, 'r') as f:
    unique_ids = json.load(f)

for id in unique_ids:
    for i, track in enumerate(input_data):
        if id == track["track_id"]:
            break
        if i == len(input_data) - 1:
            tmp = {
                "track_id": id,
                "play_count_week_1": 0,
                "play_count_week_2": 0,
                "play_count_week_3": 0,
                "play_count_week_4": 0,
                "play_count": 0
                }
            input_data.append(tmp)

data_weeks = pd.DataFrame(input_data)
data_weeks["plays"] = (data_weeks["play_count_week_1"] +
                       data_weeks["play_count_week_2"] +
                       data_weeks["play_count_week_3"] +
                       data_weeks["play_count_week_4"])
data_weeks.sort_values(by=["track_id"], inplace=True, ascending=True)
data_weeks.drop(columns=["track_id", "play_count_week_1", "play_count_week_2",
                         "play_count_week_3", "play_count_week_4", "play_count"],
                inplace=True)

with open(output_path, 'r') as f:
    output_data = json.load(f)


def generate_ranking(output_data: List[float],
                     data_weeks: pd.DataFrame) -> None:
    data_weeks = data_weeks.values.tolist()
    tmp_df = []
    for row in data_weeks:
        tmp_df.extend(row)
    data_weeks = tmp_df

    for i in range(len(output_data)):
        output_data[i] = (output_data[i] + data_weeks[i])

    ranking = sorted(range(len(output_data)),
                     key=lambda i: output_data[i])
    ranking.reverse()

    for i, idx in enumerate(ranking[:50]):
        print(f"{i}: {idx} - {round(output_data[idx])}")

    with open(ranking_path, 'w+') as f:
        f.write(json.dumps(ranking, indent=4))


generate_ranking(output_data, data_weeks)

0: 2165 - 275
1: 1509 - 274
2: 870 - 274
3: 60 - 274
4: 1057 - 273
5: 1960 - 267
6: 1354 - 267
7: 3189 - 266
8: 1442 - 266
9: 1424 - 266
10: 3949 - 265
11: 1383 - 262
12: 1351 - 262
13: 994 - 261
14: 2615 - 260
15: 1914 - 260
16: 1877 - 260
17: 3358 - 259
18: 454 - 259
19: 4020 - 257
20: 2241 - 257
21: 740 - 257
22: 333 - 257
23: 1234 - 256
24: 1097 - 256
25: 1639 - 255
26: 1569 - 255
27: 2902 - 253
28: 1628 - 253
29: 4030 - 252
30: 3944 - 252
31: 3195 - 252
32: 3135 - 252
33: 2182 - 250
34: 1706 - 250
35: 1526 - 250
36: 3750 - 249
37: 2913 - 249
38: 1471 - 249
39: 901 - 249
40: 853 - 248
41: 3854 - 247
42: 3510 - 247
43: 2703 - 247
44: 1380 - 247
45: 1136 - 247
46: 3457 - 246
47: 1534 - 246
48: 1111 - 246
49: 3266 - 245


Adv. model

In [6]:
actual_ranking = os.path.join(dirname,
                              "microservice/"
                              "database/"
                              "actual_ranking.json")
pred_ranking = os.path.join(dirname,
                              "microservice/"
                              "database/"
                              "adv_model/"
                              "2023-01-17_00-33-27/"
                              "ranking.json")

with open(actual_ranking, 'r') as f:
    ranking_1 = json.load(f)

with open(pred_ranking, 'r') as f:
    ranking_2 = json.load(f)


def calculate_criterion(ranking_1: List[int], ranking_2: List[int]) -> float:
    criterion = 0
    count_penalty = 0
    for i in range(50):
        index_1 = i + 1
        index_2 = ranking_2.index(ranking_1[i]) + 1
        if index_2 == index_1:
            count_penalty += 1
        criterion += ((index_1 - index_2) ** 2) / (i + 1)
    if count_penalty < 40:
        criterion *= 10
    return criterion


print(calculate_criterion(ranking_1, ranking_2))

4694.139696753117


Base model

In [7]:
actual_ranking = os.path.join(dirname,
                              "microservice/"
                              "database/"
                              "actual_ranking.json")
pred_ranking = os.path.join(dirname,
                              "microservice/"
                              "database/"
                              "base_model/"
                              "2023-01-17_02-12-19/"
                              "ranking.json")

with open(actual_ranking, 'r') as f:
    ranking_1 = json.load(f)

with open(pred_ranking, 'r') as f:
    ranking_2 = json.load(f)


def calculate_criterion(ranking_1: List[int], ranking_2: List[int]) -> float:
    criterion = 0
    count_penalty = 0
    for i in range(50):
        index_1 = i + 1
        index_2 = ranking_2.index(ranking_1[i]) + 1
        if index_2 == index_1:
            count_penalty += 1
        criterion += ((index_1 - index_2) ** 2) / (i + 1)
    if count_penalty < 40:
        criterion *= 10
    return criterion


print(calculate_criterion(ranking_1, ranking_2))

74818.14208782068


Po porównaniu wyników modeli wyraźnie widać, że model docelowy jest znacznie lepszy od modelu bazowego (wartość naszej metryki jest dla niego ~16 razy mniejsza).
Niestety nie udało nam się spełnić kryterium analitycznego z Etapu I, które zakładało, że wartość tej metryki zejdzie ponieżej `0.2`. Założenie to wynikało z niedocenienia złożoności problemu, który przyszło nam rozwiązywać - taką wartość osiągnąć możnaby było tylko dla co najwyżej kilku błędnych pozycji w top liście (i to na zasadzie zamiany miejsc po sąsiedzku, np. faktyczne pozycje 16 i 17 w naszej predykcji byłby pozycjami 17 i 16 - w rzeczywistości wystarczy, że pozycja jednej piosenki jest np. 10 miejsc dalej niż powinna i już wartość metryki znacznie wykracza poza zakładane kryterium).\
Również kryterium biznesowe (40/50 pozycji poprawnych) nie zostało spełnione, co tłumaczy wspomniane powyżej zjawisko (wystarczy, że 1 piosenka z 50 jest odpowiednio nietrafiona).